In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
import numpy as np 
import pandas as pd 


In [2]:
train_data = pd.read_csv('usjobs_train.csv')
test_data = pd.read_csv('usjobs_test.csv')

In [3]:
train_data.sample(5)

# randomly selecting a sample of 5 data points


,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,...,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL
16148,job_2a7a50622c24a7ce,Data Scientist - Trust and Safety Product - USDS,Data Scientist,NaN,NaN,TikTok,"Los Angeles, CA+38 locations",Los Angeles,CA,year,...,"['PowerPoint', 'SQL', 'Python']",Audiovisual and Media Communications,Media Communications,NaN,XXXL,3.4,122.0,Shou Zi Chew,0.68,https://www.tiktok.com/
31565,job_4b423846f9afdebe,Business System Analyst,Business Analyst,NaN,NaN,Buzzclan LLC,"Hayward, CA 94545 (Mt Eden area)+1 location",Hayward,CA,hour,...,[],Information Technology,Information Technology,XXS,XS,2.3,4.0,NaN,NaN,http://www.buzzclan.com/
7126,job_9a45afdc8c75371f,Business Analyst,Business Analyst,NaN,Remote,Booz Allen Hamilton,"Melbourne, FL+9 ubicaciones",Melbourne,FL,year,...,"['Agile', 'Bachelor']",Management and Consulting,Consulting,XXL,XXXL,3.9,2464.0,Horacio D. Rozanski,0.87,http://www.boozallen.com/
12226,job_6cc9a812b16ed2db,Assistant Controller,Controller,NaN,Remote,Central Oregon Community College,"Bend, OR 97701 (River West area)",Bend,OR,year,...,"['English', 'Spanish', 'Master', 'Word', 'Acce...",Education,Education,S,L,4.0,58.0,NaN,NaN,http://www.cocc.edu/
31498,job_caec93625b745c26,"Decision Scientist, Medicare Analytics",Data Scientist,NaN,NaN,CVS Health,"New York, NY+2 locations",New York,NY,year,...,"['PowerPoint', 'Python', 'MBA', 'Master', 'SQL...",Healthcare,Health,XXXL,XXXL,3.2,43596.0,"Karen S. Lynch, President & CEO",0.59,http://jobs.cvshealth.com/


In [4]:
test_data.isna().sum()

# identify and count the missing values (NaN or null values) in the test dataset

ID                      0
Job                     0
Jobs_Group              0
Profile             14061
Remote              12822
Company                10
Location                8
City                 2625
State                2119
Frecuency_Salary        0
Skills                  0
Sector               4735
Sector_Group         4735
Revenue             12022
Employee             8418
Company_Score        5828
Reviews              5828
Director            13661
Director_Score      14464
URL                 10519
dtype: int64

In [5]:
ID = test_data['ID']

In [6]:
def preprocessing(_data_):
    
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor        # ensemble learning method for regression tasks
    from sklearn.impute import SimpleImputer                  # imputing missing values in a dataset
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import mean_squared_error            # evaluate the performance of regression models
    from sklearn.preprocessing import LabelEncoder            # convert categorical labels into numerical labels
    from sklearn.preprocessing import MinMaxScaler            # scales features to a specified range (usually between 0 and 1)

    
    
    
    #Removes duplicate rows based on the 'ID' and 'Job' columns
    
    _data_.drop_duplicates(subset=['ID', 'Job'], inplace=True)  #inplace=True (changes should be made directly to the original DataFrame)
    
    
    
    # Drops unnecessary columns from the dataset
    
    _data_.drop(['ID','Job','URL','Director','State','City','Company','Sector','Location'],inplace = True,axis = 1)
    
    
    
    # Fills missing values in 'Profile', 'Remote', 'Sector_Group', and 'Revenue' columns with specified default values
    
    # train_data.dropna(subset=['Location'],inplace = True)
    _data_['Profile'].fillna('Not Specified', inplace=True)
    _data_['Remote'].fillna('Not Specified', inplace=True)
    
    
    # Fill null values in 'Sector_Group' with 'Not Specified'
    _data_['Sector_Group'].fillna('Not Specified', inplace=True)
    
    

    # Categorical Data Processing

    # Define a threshold for considering significant categories
    threshold = 800

    # creating a list of categories in the 'Sector_Group' column that occur less frequently than specified threshold
    infrequent_categories = _data_['Sector_Group'].value_counts()[_data_['Sector_Group'].value_counts() < threshold].index

    # Replace infrequent categories with 'Others'
    _data_['Sector_Group'] = np.where(_data_['Sector_Group'].isin(infrequent_categories), 'Others', _data_['Sector_Group'])
    
    # Impute missing values in the 'Revenue' column with the most common category ('XXXL')
    _data_['Revenue'].fillna('XXXL', inplace=True)

    # Group categories into broader ranges
    revenue_mapping = {
        'XXXS': 'Low',
        'XXS': 'Low',
        'XS': 'Low',
        'S': 'Medium',
        'M': 'Medium',
        'L': 'Medium',
        'XL': 'High',
        'XXL': 'High',
        'XXXL': 'High'
    }

    _data_['Revenue_Grouped'] = _data_['Revenue'].map(revenue_mapping)
    
        
    # Impute missing values in the 'Employee' column with the most common category ('XXXL')
    _data_['Employee'].fillna('XXXL', inplace=True)

    # Group categories into broader ranges
    employee_mapping = {
        'XXXS': 'Small',
        'XXS': 'Small',
        'XS': 'Small',
        'S': 'Medium',
        'M': 'Medium',
        'L': 'Large',
        'XL': 'Large',
        'XXL': 'Large',
        'XXXL': 'Large'
    }

    _data_['Employee_Grouped'] = _data_['Employee'].map(employee_mapping)



    # Extract rows with missing 'Company_Score' values for prediction
    X_test = _data_[pd.isnull(_data_['Company_Score'])][['Company_Score']]
    y_train = _data_.dropna(subset=['Company_Score'])['Company_Score']

    # Use only 'Company_Score' for training
    X_train = y_train.to_frame()

    # a pipeline with imputation and the model
    model = make_pipeline(
        SimpleImputer(strategy='mean'),
        RandomForestRegressor()
    )

    # Train the model
    model.fit(X_train, y_train)

    # predictions on the test set
    predicted_scores = model.predict(X_test)

    # Impute missing values with predicted scores
    _data_.loc[pd.isnull(_data_['Company_Score']), 'Company_Score'] = predicted_scores

    # Mean Squared Error on the training set
    train_predictions = model.predict(X_train)
    train_mse = mean_squared_error(y_train, train_predictions)

    print(f'Training MSE: {train_mse}')




    # Extract rows with missing 'Review' values for prediction
    X_test_review = _data_[pd.isnull(_data_['Reviews'])][['Reviews']]
    y_train_review = _data_.dropna(subset=['Reviews'])['Reviews']

    # Use only 'Review' for training
    X_train_review = y_train_review.to_frame()

    # Create a pipeline with imputation and the model
    model_review = make_pipeline(
        SimpleImputer(strategy='mean'),
        RandomForestRegressor()
    )

    # Train the model
    model_review.fit(X_train_review, y_train_review)

    # Make predictions on the test set
    predicted_reviews = model_review.predict(X_test_review)

    # Impute missing values with predicted scores
    _data_.loc[pd.isnull(_data_['Reviews']), 'Reviews'] = predicted_reviews

    # Calculate Mean Squared Error on the training set
    train_predictions_review = model_review.predict(X_train_review)
    train_mse_review = mean_squared_error(y_train_review, train_predictions_review)

    print(f'Training MSE for Reviews: {train_mse_review}')





    # Extract rows with missing 'Director_Score' values for prediction
    X_test_director_score = _data_[pd.isnull(_data_['Director_Score'])][['Company_Score', 'Reviews']]
    y_train_director_score = _data_.dropna(subset=['Director_Score'])['Director_Score']

    # Use only 'Director_Score', 'Company_Score', and 'Reviews' for training
    X_train_director_score = _data_.dropna(subset=['Director_Score'])[['Company_Score', 'Reviews']]

    # Create a pipeline with imputation and the model
    model_director_score = make_pipeline(
        SimpleImputer(strategy='mean'),
        RandomForestRegressor()
    )

    # Train the model
    model_director_score.fit(X_train_director_score, y_train_director_score)

    # Make predictions on the test set
    predicted_director_scores = model_director_score.predict(X_test_director_score)

    # Impute missing values with predicted scores
    _data_.loc[pd.isnull(_data_['Director_Score']), 'Director_Score'] = predicted_director_scores

    # Calculate Mean Squared Error on the training set
    train_predictions_director_score = model_director_score.predict(X_train_director_score)
    train_mse_director_score = mean_squared_error(y_train_director_score, train_predictions_director_score)

    print(f'Training MSE for Director_Score: {train_mse_director_score}')



    _data_.drop(['Revenue','Employee'],axis = 1,inplace = True)

    _data_categorical = _data_[_data_.select_dtypes(include=['object']).columns]
    _data_numerical = _data_[_data_.select_dtypes(exclude=['object']).columns]

    _data_categorical['Skills'] = _data_categorical['Skills'].apply(eval)

    # Get the unique skills
    all_skills = set(skill for skills_list in _data_categorical['Skills'] for skill in skills_list)

    # Create binary columns for each skill and initialize with zeros
    for skill in all_skills:
        _data_categorical[skill] = 0

    # Update the binary columns based on the presence of each skill in the 'Skills' column
    for index, row in _data_categorical.iterrows():
        skills_list = row['Skills']
        for skill in skills_list:
            _data_categorical.at[index, skill] = 1

    # Drop the original 'Skills' column
    _data_categorical.drop('Skills', axis=1,inplace = True)




    enc = LabelEncoder()
    _data_categorical['Jobs_Group'] = enc.fit_transform(_data_categorical['Jobs_Group'])
    _data_categorical['Sector_Group'] = enc.fit_transform(_data_categorical['Sector_Group'])



    profile_mapping = {'Lead': 1, 'Senior': 2, 'Junior': 3,'Not Specified': 0}
    remote_mapping = {'Remote': 1, 'Hybrid': 2, 'Not Specified': 0}
    frequency_mapping = {'hour': 0, 'day': 1, 'week': 2, 'month': 3 , 'year': 4}
    employee_mapping = {'Small' : 0,'Medium': 1,'Large': 2}
    revenue_mapping = {'Low' : 0,'Medium': 1,'High': 2}

    _data_categorical['Profile'] = _data_categorical['Profile'].map(profile_mapping)
    _data_categorical['Remote'] = _data_categorical['Remote'].map(remote_mapping)
    _data_categorical['Frecuency_Salary'] = _data_categorical['Frecuency_Salary'].map(frequency_mapping)
    _data_categorical['Employee_Grouped'] = _data_categorical['Employee_Grouped'].map(employee_mapping)
    _data_categorical['Revenue_Grouped'] = _data_categorical['Revenue_Grouped'].map(revenue_mapping)
    
    


    def scale_dataframe(df):
        # Select only numerical columns
        numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

        # Create a MinMaxScaler
        scaler = MinMaxScaler()

        # Scale the numerical columns
        df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

        return df
    
    scaled_data_categorical = scale_dataframe(_data_categorical)
    scaled_data_numerical = scale_dataframe(_data_numerical)
    
    
    preprocessed_data = pd.concat([scaled_data_categorical, scaled_data_numerical], axis=1)
    
    return preprocessed_data

In [7]:
preprocessed_train_data = preprocessing(train_data)
preprocessed_test_data = preprocessing(test_data)

Training MSE: 2.8587764436738566e-10
Training MSE for Reviews: 25.770179527076902
Training MSE for Director_Score: 0.0005357843439898838


C:\Users\HP\AppData\Local\Temp\ipykernel_2280\1985842077.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data_categorical['Skills'] = _data_categorical['Skills'].apply(eval)
C:\Users\HP\AppData\Local\Temp\ipykernel_2280\1985842077.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data_categorical[skill] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_2280\1985842077.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

Training MSE: 8.036479373238199e-08
Training MSE for Reviews: 103.70498601419932
Training MSE for Director_Score: 0.0006065313264263027


C:\Users\HP\AppData\Local\Temp\ipykernel_2280\1985842077.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data_categorical['Skills'] = _data_categorical['Skills'].apply(eval)
C:\Users\HP\AppData\Local\Temp\ipykernel_2280\1985842077.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data_categorical[skill] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_2280\1985842077.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [8]:
preprocessed_train_data.shape

(33248, 109)

In [9]:
preprocessed_test_data.shape

(22166, 108)

In [10]:
preprocessed_train_data

,Jobs_Group,Profile,Remote,Frecuency_Salary,Sector_Group,Revenue_Grouped,Employee_Grouped,Kaggle,Looker,Streamlit,...,Qlik,Snaplogic,Fabric,Google Sheets,Essbase,SciKit,Mean_Salary,Company_Score,Reviews,Director_Score
0,9,0.000000,0.5,0.0,9,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.183744,0.800,0.000077,0.895594
1,4,0.000000,0.0,1.0,9,1.0,0.5,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.305314,0.625,0.000230,0.680851
2,9,0.666667,0.0,1.0,11,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.130775,0.600,0.000351,0.586596
3,2,0.666667,1.0,1.0,2,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.177883,0.700,0.000117,0.680851
4,11,0.000000,0.5,0.0,3,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.162366,0.800,0.000746,0.840426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33243,8,0.000000,0.0,1.0,3,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.066006,0.650,0.000149,0.654469
33244,8,0.000000,0.0,1.0,3,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.122510,0.575,0.000117,0.744681
33245,9,0.000000,0.0,1.0,3,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.192268,0.825,0.018494,0.902979
33246,1,0.000000,0.0,0.0,6,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.183744,0.675,0.010979,0.590713


In [11]:
preprocessed_test_data

,Jobs_Group,Profile,Remote,Frecuency_Salary,Sector_Group,Revenue_Grouped,Employee_Grouped,Kaggle,Looker,Streamlit,...,Java,Qlik,Snaplogic,Fabric,Google Sheets,Essbase,SciKit,Company_Score,Reviews,Director_Score
0,1,0.333333,0.5,1.0,7,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000,0.000004,0.908293
1,9,0.666667,0.0,1.0,7,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675,0.010446,0.541500
2,10,0.666667,1.0,1.0,6,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675,0.010446,0.541500
3,7,0.000000,0.5,1.0,7,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750,0.001105,0.731707
4,9,0.000000,0.0,1.0,6,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675,0.010446,0.541500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22161,4,0.000000,0.5,1.0,7,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750,0.000117,0.793077
22162,9,0.666667,0.0,1.0,7,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600,0.006555,0.597561
22163,0,0.000000,0.0,0.0,4,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675,0.010446,0.541500
22164,7,0.000000,1.0,1.0,0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750,0.002900,0.817073


In [12]:
# Model training

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

#
X = preprocessed_train_data.drop('Mean_Salary', axis=1)
y = preprocessed_train_data['Mean_Salary']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()


with tqdm(total=100, desc="Training") as pbar:
    model.fit(X_train, y_train)
    pbar.update(50)  
    pbar.set_postfix({"Epoch": 1})  


with tqdm(total=100, desc="Predicting") as pbar:
    y_pred = model.predict(X_test)
    pbar.update(50)  

    
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")


Predicting:  50%|█████     | 50/100 [00:00<00:00, 67.68it/s]

Mean Squared Error: 0.0029384052393772037
R-squared Score: 0.5008422250492962


In [14]:
preprocessed_test_data.columns

Index(['Jobs_Group', 'Profile', 'Remote', 'Frecuency_Salary', 'Sector_Group',
       'Revenue_Grouped', 'Employee_Grouped', 'Kaggle', 'Looker', 'Streamlit',
       ...
       'Java', 'Qlik', 'Snaplogic', 'Fabric', 'Google Sheets', 'Essbase',
       'SciKit', 'Company_Score', 'Reviews', 'Director_Score'],
      dtype='object', length=108)

In [15]:
preprocessed_test_data.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [16]:
preprocessed_test_data = preprocessed_test_data[X_test.columns]

In [18]:
import pickle

with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [19]:
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

y_pred_test = loaded_model.predict(preprocessed_test_data)

predictions_df = pd.DataFrame({'ID':ID, 'Predicted_Target': y_pred_test})

print(predictions_df)


                         ID  Predicted_Target
0       sj_99ad4f80ae7f4835          0.201234
1      job_6ff7f1a7c400916a          0.154099
2      job_e059d20eba88b17a          0.237814
3      job_79f7953813b13358          0.205652
4      job_892dfe86b96f322b          0.124135
...                     ...               ...
22161  job_4ab2e5201b60cc15          0.156996
22162  job_b3a7cf40a22659ef          0.149465
22163  job_4e69acaf903b0276          0.126640
22164  job_816f279ea500ecd0          0.215244
22165  job_9861d87455c4a6a6          0.258095

[22166 rows x 2 columns]


In [21]:
predictions_df[predictions_df['ID'] == "sj_99ad4f80ae7f4835"]

,ID,Predicted_Target
0,sj_99ad4f80ae7f4835,0.201234
